# TopicModelling- Vorbereitung

Um das Topic-Suchen zu ermöglichen laden wir zunächst BERTopic. Es handelt sich hier um ein vortrainiertes Model, dass sich am besten für unsere Zwecke eignet.
Alles zu Berttopic:
@article{grootendorst2022bertopic,
  title={BERTopic: Neural topic modeling with a class-based TF-IDF procedure},
  author={Grootendorst, Maarten},
  journal={arXiv preprint arXiv:2203.05794},
  year={2022}
}

In [1]:
pip install bertopic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
pip install upgrade bertopic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement upgrade (from versions: none)
ERROR: No matching distribution found for upgrade


In [3]:
!pip install joblib==1.1.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from bertopic import BERTopic

In [5]:
pip install bertopic[visualization]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

Wir nutzen ein Pretrained Model: https://www.sbert.net/docs/pretrained_models.html

Leider ist beim zip und entzippen ein Fehler unterlaufen, den wir nicht nachvollziehen können. Alle Spalten sind als Strings gespeichert. Wenn aber das Erbenis des PreProcessings eine leere Zeile ist, wir diese als Float in der csv/zip gespeichert. Deshalb müssen wir zunächst alle float-Zeilen wieder leeren. Das Model kann keine Float-Zeilen lesen.

In [7]:
bereinigtwesten=pd.read_csv('westen_prepro.zip')
for i in range(0,len(bereinigtwesten["rejoined"])):
  if type(bereinigtwesten["rejoined"][i])==float:
    bereinigtwesten["rejoined"][i]=''
datawest=bereinigtwesten["rejoined"]

In [8]:
bereinigtosten=pd.read_csv('osten_prepro.zip')

for i in range(0,len(bereinigtosten["rejoined"])):
  if type(bereinigtosten["rejoined"][i])==float:
    bereinigtosten["rejoined"][i]=''
dataost=bereinigtosten["rejoined"]

# Model  (ohne Parameter, ohne FineTune)


Wir nutzen ein sentence-transformer. Dieses fasst jeden einzelen Satz/Tweets in einem Vektor-Space (384 Dim) zusammen. Somit bleiben Sinn und Kohärenz bestehen. Das Model ist für viele Zwecke geeignet und somit passend für unser Thema.
https://www.sbert.net/docs/pretrained_models.html

Im ersten Testversuch übergeben mit dem Model keine Parameter, sonder nur die Tweets und das Model.

In [9]:
topic_model=BERTopic(embedding_model='all-MiniLM-L12-v2')

In [10]:
topics, probs= topic_model.fit_transform(datawest)

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/573 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/352 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

Das Ergebnis zeigt, dass wir über 500 verschiedene Themen haben. Also kein besonders gut zu gebrauchenes Ergebnis. Deshalb müssen wir am dem Model arbeiten.

In [11]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,6580,-1_stadt_ukrain_kinder_tschenp
1,0,658,0_land_landesgesetz_landesregierung_landesfina...
2,1,465,1_informationen_informieren_info_finden
3,2,390,2_video_videokonferenz_videoreih_erzählt
4,3,382,3_europa_europäischen_europawahl_europäisch
...,...,...,...
539,538,10,538_ls_hallo_dankeschön_kirstiemorrow
540,539,10,539_norddeutschland_alexi_nordsyrien_frühstück...
541,540,10,540_ablehnung_einzelplan_stimmen_angenommen
542,541,10,541_video_bürgerschaft_onlin_hbbue


Ein Blick in die einzelnen Themen zeigt aber, dass das Training gar nicht mal soo schlechte Ergebnisse geliefert hat.

In [12]:
topic_model.get_topic(1) 

[('informationen', 0.01938782999993834),
 ('informieren', 0.015474233956346999),
 ('info', 0.015417004570338037),
 ('finden', 0.011560042371805107),
 ('hauptausschuss', 0.00695719740761767),
 ('aghohoho', 0.006305221895115627),
 ('anhörung', 0.006113196710289241),
 ('weiter', 0.0058717226684734825),
 ('sobald', 0.005768793759739464),
 ('inform', 0.005597632913011406)]

In [13]:
topic_model.get_topic(5) 

[('neuinfektionen', 0.09093195020518316),
 ('gemeldet', 0.09068006114644304),
 ('zahlen', 0.07141944777095702),
 ('gestern', 0.060827981498871156),
 ('weiter', 0.05925443948142331),
 ('wurden', 0.05662263117155509),
 ('informationen', 0.048641291421560484),
 ('seit', 0.04321952553988385),
 ('informatinen', 0.0033571931213675025),
 ('tanzveranstaltungen', 0.0016785965606837512)]

In [14]:
topic_model.get_topic(10) 

[('demokrati', 0.04294525573523313),
 ('herzkamm', 0.014212756510386377),
 ('demokratieprei', 0.014005592866355623),
 ('politik', 0.013800074997881604),
 ('demokratischen', 0.013453105129525302),
 ('demokratiebildung', 0.010888327924307824),
 ('demokratisch', 0.010009071456272299),
 ('dasbestewaswirhaben', 0.009917634319106566),
 ('demokratieleuchtet', 0.009710725544130624),
 ('demokraten', 0.008901904687611206)]

# Model 1 mit Tuning

Im ersten Fine-Tuning-Schritt veränder wir 2 Parameter.
top_n_words: Wir vergrößern die Anzahl der Wörter, die pro Thema zugeordnet werden. Nach meherern Durchgängen hat das Ergebnis mit 23 Wörtern pro Thema am besten abgeschnitten.
n_gram_range: Wir lassen zu, dass auch Wortpaare (max. 2 Wörter) zusammengehören und als eine Einheit gelten. Hierfür müssen sie aber auffällig oft nebeneinander stehen.

In [15]:
topic_model1=BERTopic(embedding_model='all-MiniLM-L12-v2', top_n_words=23, n_gram_range=(1,2))

In [16]:
topics, probs= topic_model1.fit_transform(datawest)

Wir sehen, dass sich an der Themenanzahl wenig verändert hat, und an dern einzelnen Wörter des Themas auch nicht. Deshalb müssen wir noch einen Schritt weiter gehen.

In [17]:
topic_model1.get_topic_info()

,Topic,Count,Name
0,-1,6810,-1_demokrati_info_the_land
1,0,620,0_land_landesgesetz_landesgesetz änderung_land...
2,1,441,1_europa_europäischen_europäisch_europawahl
3,2,412,2_video_videokonferenz_per videokonferenz_per
4,3,359,3_weiter zahlen_zahlen seit_neuinfektionen gem...
...,...,...,...
542,541,10,541_kampagn_kriegsgräberfürsorg_inhaltlich kam...
543,542,10,542_präsidium_mitglied präsidium_präsidenten_l...
544,543,10,543_winter_winternotprogramm_umweltsen jensker...
545,544,10,544_bundesregelung_bundesrat_irgendwelchen_sie...


In [18]:
topic_model1.get_topic(1) 

[('europa', 0.039166270762561246),
 ('europäischen', 0.01906961982166505),
 ('europäisch', 0.014784410177889059),
 ('europawahl', 0.012610553034149787),
 ('zukunft europa', 0.010630415830591438),
 ('europaausschuss', 0.010526325599612944),
 ('blick europa', 0.00903726650113695),
 ('eu', 0.00844486824634762),
 ('europawoch', 0.007477388875702338),
 ('europaminist', 0.006890128397360304),
 ('zukunft', 0.006874758466319349),
 ('konferenz zukunft', 0.006825414923155664),
 ('konferenz', 0.006806181971684584),
 ('kolumn', 0.006370387261611953),
 ('kolumn blick', 0.0061057229241578736),
 ('union', 0.005747800846756279),
 ('blick', 0.005379168086410661),
 ('weilwichtig', 0.00531182688200398),
 ('europakonferenz', 0.004696709941659903),
 ('regionen', 0.004355004370116316),
 ('landesparlament', 0.0043158593416282335),
 ('deutschland', 0.004171583514471926),
 ('europaweit', 0.0041464884281201)]

# Model 2 mit UMAP


Auch hier durch Testen auch min_dist=0.3.
UMAP Quelle: https://umap-learn.readthedocs.io/en/latest/parameters.html

In [19]:
from umap import UMAP

umap_model = UMAP(n_neighbors=20, n_components=10, metric='cosine', min_dist=0.3, low_memory=False)
topic_model2 = BERTopic(embedding_model='all-MiniLM-L12-v2', top_n_words=23, n_gram_range=(1,2), umap_model=umap_model).fit(datawest)

Wir haben schon weniger Topics und die dazugehörigen Worte passen auch.

In [20]:
topic_model2.get_topic_info()

,Topic,Count,Name
0,-1,10970,-1_info_demokrati_menschen_parlament
1,0,816,0_land_landesgesetz_landesregierung_landesgese...
2,1,363,1_kinder_kindern_jugendlich_kinder jugendlich
3,2,363,2_weiter neuinfektionen_neuinfektionen gemelde...
4,3,333,3_video_videokonferenz_per videokonferenz_ganz...
...,...,...,...
420,419,10,419_einzelhändl_einzelhandel_sicher einzelhänd...
421,420,10,420_astridwallmann_lpin astridwallmann_lpin_as...
422,421,10,421_kretschmann_herr kretschmann_herr_funktion
423,422,10,422_eröffnung_ausstellung_ausstellungseröffnun...


In [21]:
topic_model1.get_topic(1) 

[('europa', 0.039166270762561246),
 ('europäischen', 0.01906961982166505),
 ('europäisch', 0.014784410177889059),
 ('europawahl', 0.012610553034149787),
 ('zukunft europa', 0.010630415830591438),
 ('europaausschuss', 0.010526325599612944),
 ('blick europa', 0.00903726650113695),
 ('eu', 0.00844486824634762),
 ('europawoch', 0.007477388875702338),
 ('europaminist', 0.006890128397360304),
 ('zukunft', 0.006874758466319349),
 ('konferenz zukunft', 0.006825414923155664),
 ('konferenz', 0.006806181971684584),
 ('kolumn', 0.006370387261611953),
 ('kolumn blick', 0.0061057229241578736),
 ('union', 0.005747800846756279),
 ('blick', 0.005379168086410661),
 ('weilwichtig', 0.00531182688200398),
 ('europakonferenz', 0.004696709941659903),
 ('regionen', 0.004355004370116316),
 ('landesparlament', 0.0043158593416282335),
 ('deutschland', 0.004171583514471926),
 ('europaweit', 0.0041464884281201)]

# Model 3 HDBSCAN

Hier werden einzelne Topics so einem Cluster verbunden, also nochmal Eingrenzung der Topics. Wir versuchen auch hier weiterhin das Rauschen zu mindern.

In [22]:
from hdbscan import HDBSCAN 

hdbscan_model = HDBSCAN(min_cluster_size=35, min_samples=5, metric='euclidean', prediction_data=True)
topic_model3 = BERTopic(embedding_model='all-MiniLM-L12-v2', top_n_words=23, n_gram_range=(1,2), umap_model=umap_model, hdbscan_model=hdbscan_model).fit(datawest)

euclidean-metric: https://www.analyticsvidhya.com/blog/2020/02/4-types-of-distance-metrics-in-machine-learning/

Entscheidung gegegn k-Clustering, weil sonst jedes wort in einem bestimmten Cluster vorkoommen muss und das kann unsinnig werden.

In [23]:
topic_model3.get_topic_info()

,Topic,Count,Name
0,-1,11274,-1_hbbue_info_debatt_menschen
1,0,917,0_land_landesgesetz_landesregierung_landesgese...
2,1,571,1_informationen_info_finden_informationen finden
3,2,548,2_demokrati_politik_politisch_polizei
4,3,486,3_europa_europäischen_europaausschuss_europäisch
...,...,...,...
183,182,36,182_hall_ganztägig beratung_ganztägig_ihk
184,183,36,183_parlamentarisch demokrati_demokrati_buch p...
185,184,36,184_anfragen stadtbürgerschaft_stadtbürgerscha...
186,185,35,185_ausgewählt nächsten_nächsten woch_woch aus...


# Model 4 (update 3) Count Vektorizer

min_df: gibt an wie oft ein Wort min. vorkommen muss, um überhaupt als repräsentativ zu gelten.

In [24]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer_model = CountVectorizer(min_df=15)

topic_model3.update_topics(datawest, vectorizer_model=vectorizer_model)

In [25]:
topic_model3.get_topic_info()

,Topic,Count,Name
0,-1,11274,-1_debatt_menschen_tschenp_stadt
1,0,917,0_land_landesregierung_bundesrat_änderung
2,1,571,1_informationen_info_finden_homepag
3,2,548,2_demokrati_politik_polizei_politisch
4,3,486,3_europa_eu_konferenz_zukunft
...,...,...,...
183,182,36,182_ausbildung_vorbei_beratung_kommt
184,183,36,183_demokrati_parlamentarisch_schreibt_informa...
185,184,36,184_anfragen_onlin_hbbue_nächsten
186,185,35,185_nächsten_woch_nächste_hoch


# Model 5 c-TIF-IDF

Hat keine besseren Ergebnisse geliefert und wird deshalb hier weggelassen. Im Ursprünglichen Model ist diese Funktion schon integriergt und kann deshalb vernachlässigt werden.

In [26]:
#from bertopic.vectorizers import ClassTfidfTransformer

#ctfidf_model = ClassTfidfTransformer(bm25_weighting=True)
#topic_model5 = BERTopic(embedding_model='all-MiniLM-L12-v2', top_n_words=23, n_gram_range=(1,2), umap_model=umap_model, hdbscan_model=hdbscan_model, ctfidf_model=ctfidf_model, vectorizer_model=vectorizer_model).fit(datawest)

In [27]:
#topic_model5.get_topic_info()

#Versuch der Themenreduzierung

Leider lässt sich die Themenmenge nicht sinnvoll reduzieren. Es lassen sich lediglich einzelne Themen zusammenfassen (merge).

In [28]:
#topic_model6 = BERTopic(embedding_model='all-MiniLM-L12-v2', top_n_words=30, n_gram_range=(1,2), nr_topics=300, umap_model=umap_model, hdbscan_model=hdbscan_model, vectorizer_model=vectorizer_model).fit(datawest)

In [29]:
#topic_model6.get_topic_info()

In [30]:
#new_topics, new_probs = topic_model2.reduce_topics(datawest, nr_topics=180)

In [31]:
#topic_model7 = BERTopic(embedding_model='all-MiniLM-L12-v2', top_n_words=30, n_gram_range=(1,2), nr_topics=100, umap_model=umap_model, hdbscan_model=hdbscan_model, vectorizer_model=vectorizer_model).fit(datawest)

In [33]:
#topic_model7.get_topic_info()

# Model Final: West

In [46]:
umap_model = UMAP(n_neighbors=20, n_components=10, metric='cosine',min_dist=0.3, low_memory=False)
hdbscan_model = HDBSCAN(min_cluster_size=35, min_samples=5, metric='euclidean', prediction_data=True)
vectorizer_model = CountVectorizer(min_df=15)                     

In [47]:
topic_model_w=BERTopic(embedding_model='all-MiniLM-L12-v2', top_n_words=23, n_gram_range=(1,2), umap_model=umap_model, hdbscan_model= hdbscan_model,  vectorizer_model=vectorizer_model).fit(datawest)

In [48]:
topic_model_w.get_topic_info()

,Topic,Count,Name
0,-1,11101,-1_debatt_menschen_stadt_tschenp
1,0,1028,0_land_landesregierung_bundesrat_bund
2,1,685,1_demokrati_politisch_politik_polizei
3,2,530,2_informationen_info_informieren_finden
4,3,444,3_europa_eu_konferenz_zukunft
...,...,...,...
187,186,37,186_wandel_fahren_winnehermann_unterstützt
188,187,36,187_investitionen_kita_kommunalen_förderung
189,188,36,188_anfragen_onlin_hbbue_nächste
190,189,35,189_diskutieren_anmeldung_projekt_stellen


# Model Final: Ost

In [44]:
vectorizer_model_o = CountVectorizer(min_df=5)

topic_model_o=BERTopic(embedding_model='all-MiniLM-L12-v2', top_n_words=23, n_gram_range=(1,2), umap_model=umap_model, hdbscan_model= hdbscan_model,  vectorizer_model=vectorizer_model_o).fit(dataost)

In [45]:
topic_model_o.get_topic_info()

,Topic,Count,Name
0,-1,4402,-1_saxlt_dank_land_innen
1,0,565,0_demokrati_politik_parlament_freistaat
2,1,245,1_land_landzumleben_landesregierung_aufarbeitung
3,2,234,2_ltbb_finden_woch_redebeiträg
4,3,227,3_europa_eu_ukrain_krieg
...,...,...,...
84,83,26,83_einschränkungen_reisen_trotzdem_verschärfen
85,84,26,84_handwerk_handlungsfähig_hand_beruf
86,85,25,85_the_lithuaniamfa_ralfkalich_of
87,86,25,86_internationalen_gewalt_international_frauen


# Visualisierung

In [64]:
topic_model_w.visualize_topics()


In [65]:
topic_model_o.visualize_topics()


In [75]:
topic_model_w.visualize_barchart(top_n_topics=20)

In [76]:
topic_model_o.visualize_barchart(top_n_topics=20)

Da in unserem DataFrame sowohl Datum, als auch Uhrzeit gespeichert sind, müssen wir für die Topics-over-Time-Darstellung die Uhrzeit löschen und das Datum als reines Datum angeben.

In [51]:
bereinigtwesten['Datum'] = pd.to_datetime(bereinigtwesten['Datum']).dt.date
bereinigtosten['Datum'] = pd.to_datetime(bereinigtosten['Datum']).dt.date

In [84]:
tweets=dataost.tolist()
timestamps=bereinigtosten.Datum.tolist()
#topic_model_o.get_topic(0)
topics_over_time_o = topic_model_o.topics_over_time(tweets, timestamps)

In [96]:
topic_model_o.visualize_topics_over_time(topics_over_time_o, topics=[77])

In [86]:
tweets1=datawest.tolist()
timestamps1=bereinigtwesten.Datum.tolist()
#topic_model_w.get_topic(0)

topics_over_time_w = topic_model_w.topics_over_time(tweets1, timestamps1)


In [99]:
topic_model_w.visualize_topics_over_time(topics_over_time_w, topics=[32, 15])

In [74]:
topic_model_w.visualize_hierarchy()

In [67]:
topic_model_o.visualize_hierarchy()

In [70]:
hierarchical_topics = topic_model_w.hierarchical_topics(datawest)
tree_w = topic_model_w.get_topic_tree(hierarchical_topics)
tree_w


100%|██████████| 190/190 [00:01<00:00, 175.72it/s]


'.\n├─demokrati_europa_info_land_parlament\n│    ├─demokrati_land_digital_parlamentarisch_parlament\n│    │    ├─land_parlamentarisch_landesregierung_informationen_parlament\n│    │    │    ├─land_landesregierung_bundesrat_änderung_bund\n│    │    │    │    ├─■──andrea_mindesten_winnehermann_fahren_senat ── Topic: 47\n│    │    │    │    └─land_landesregierung_bundesrat_änderung_bund\n│    │    │    │         ├─■──petit_endet_anhörung_sachverständig_kommunal ── Topic: 46\n│    │    │    │         └─■──land_landesregierung_bundesrat_bund_änderung ── Topic: 0\n│    │    │    └─parlamentarisch_informationen_parlament_corona_info\n│    │    │         ├─■──corona_wegen_regierungserklärung_zeiten_beginn ── Topic: 10\n│    │    │         └─parlamentarisch_informationen_parlament_info_finden\n│    │    │              ├─■──informationen_info_informieren_finden_weiter ── Topic: 2\n│    │    │              └─■──parlamentarisch_parlament_termin_neu_woch ── Topic: 4\n│    │    └─demokrati_digital_p

In [71]:
hierarchical_topics = topic_model_o.hierarchical_topics(dataost)
tree_o = topic_model_o.get_topic_tree(hierarchical_topics)
tree_o

100%|██████████| 87/87 [00:00<00:00, 193.37it/s]


'.\n├─demokrati_ltbb_parlament_politik_finden\n│    ├─demokrati_parlament_ltbb_politik_finden\n│    │    ├─ltbb_finden_woch_zusammenfassung_redebeiträg\n│    │    │    ├─ltbb_finden_woch_redebeiträg_zusammenfassung\n│    │    │    │    ├─■──woch_ltbb_übersicht_finden_fachausschüss ── Topic: 70\n│    │    │    │    └─■──ltbb_finden_woch_redebeiträg_zusammenfassung ── Topic: 2\n│    │    │    └─■──zusammenfassung_finden_ltbb_rein_kolleginnen ── Topic: 59\n│    │    └─demokrati_parlament_politik_informationen_ltlsa\n│    │         ├─demokrati_parlament_politik_informationen_parlamentarischen\n│    │         │    ├─demokrati_parlament_politik_parlamentarischen_freistaat\n│    │         │    │    ├─■──demokrati_politik_parlament_freistaat_tallai ── Topic: 0\n│    │         │    │    └─■──parlamentarischen_parlament_parlamentarisch_abend_abgeordneten ── Topic: 5\n│    │         │    └─informationen_info_weiter_finden_informieren\n│    │         │         ├─■──informationen_info_weiter_inform